In [2]:
!pip install transformers datasets rouge langdetect sentence-transformers

import torch.nn.functional as F
import pandas as pd
import os
import torch
import nltk
import pickle

from transformers import BertTokenizer
from torch.utils.data import random_split

import sys
sys.path.append('/content/modules')

from modules.models import SummaryModel, ModelConfig
from modules.datasets import SentenceClusterDataset, DocSplitDataset
from modules.training import SummaryTrainer

nltk.download('all')

     |████████████████████████████████| 2.9 MB 15.0 MB/s 
     |████████████████████████████████| 285 kB 71.8 MB/s 
     |████████████████████████████████| 981 kB 55.5 MB/s 
     |████████████████████████████████| 78 kB 8.4 MB/s 
     |████████████████████████████████| 56 kB 5.9 MB/s 
     |████████████████████████████████| 895 kB 54.9 MB/s 
     |████████████████████████████████| 636 kB 78.6 MB/s 
     |████████████████████████████████| 3.3 MB 57.3 MB/s 
     |████████████████████████████████| 1.3 MB 43.8 MB/s 
     |████████████████████████████████| 125 kB 95.6 MB/s 
     |████████████████████████████████| 243 kB 94.3 MB/s 
     |████████████████████████████████| 1.2 MB 55.1 MB/s 
     |████████████████████████████████| 271 kB 78.3 MB/s 
     |████████████████████████████████| 160 kB 85.3 MB/s 
  Created wheel for langdetect: filename=langdetect-1.0.9-py3-none-any.whl size=993242 sha256=63cddb535628692e1c1e4ea77f0b16775e8a05a90b9419e0e85b076a8d7851bc
  Stored in directory: /root/.cac

True

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
tokenizer = BertTokenizer.from_pretrained("bert-base-german-cased")

Downloading:   0%|          | 0.00/249k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/474k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/433 [00:00<?, ?B/s]

In [ ]:

data = pd.read_csv('/content/drive/MyDrive/export-articles-de-2021-09-08T14:45:27.515899+00:00')
data = data[['CONTENT']].dropna()

d = data.iloc[:600]
dataset = SentenceClusterDataset(tokenizer, d)

/content/modules/datasets.py:86: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  lambda x: nltk.sent_tokenize(
/content/modules/datasets.py:89: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  lambda x: [


Downloading:   0%|          | 0.00/968 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/3.76k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/645 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/122 [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/239 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/9.08M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/480 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/14.8M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
test_size = int(len(dataset) * 0.2)
train_size = len(dataset) - test_size
train_set, test_set = random_split(dataset, [train_size, test_size])
print(len(train_set))

with open('/content/drive/MyDrive/datasets/sent-cluster/experiments_train_set-2.pt', mode='wb') as f:
  pickle.dump(train_set, f)

with open('/content/drive/MyDrive/datasets/sent-cluster/experiments_test_set-2.pt', mode='wb') as f:
  pickle.dump(test_set, f)

1884


In [ ]:
#sys.modules['models'] = sys.modules['modules.models']
#sys.modules['datasets'] = sys.modules['modules.datasets']
#sys.modules['training'] = sys.modules['modules.training']

with open('/content/drive/MyDrive/datasets/sent-cluster/experiments_train_set.pt', mode='rb') as f:
  train_set = pickle.load(f)

with open('/content/drive/MyDrive/datasets/sent-cluster/experiments_test_set.pt', mode='rb') as f:
  test_set = pickle.load(f)

## Experiment 1: Embedding Layer
1. bert embedding layer, trainable
2. bert full, trainable
3. bert embedding layer, frozen
4. bert full, frozen

In [6]:
## base config

encoder_config_base = ModelConfig({
    "embedding_dim": 768,
    "max_sequence_length": 64,
    "vocab_size": 30000,
    "sequence_encoding": 'lstm', # lstm, bert_pooled
    "lstm": {
        "layers": 1,
        "bidirectional": False,
        "dropout": 0,
        "hidden_dim": 512
    }
})

decoder_config_base = ModelConfig({
    "embedding_dim": 768,
    "max_sequence_length": 64,
    "vocab_size": 30000,
    "final_dropout": 0.0,
    "lstm": {
        "layers": 1,
        "bidirectional": False,
        "dropout": 0,
        "hidden_dim": 512
    }
})

criterion1 = torch.nn.CrossEntropyLoss(ignore_index=0)
criterion2 = torch.nn.CosineSimilarity(dim=-1)
criterions = {
    'reconstruction_loss': criterion1,
    'similarity_loss': criterion2
}

In [ ]:
# 1. bert embedding layer, trainable

config_e = ModelConfig({
    "embedding_dim": 768,
    "max_sequence_length": 64,
    "vocab_size": 30000,
    "sequence_encoding": 'lstm',
    "lstm": {
        "layers": 1,
        "bidirectional": False,
        "dropout": 0,
        "hidden_dim": 512
    },
    "embeddings": {
        "mode": "bert_weights_extracted",
        "trainable": True,
        "dropout": 0.1
    }
})

config_d = ModelConfig({
    "embedding_dim": 768,
    "max_sequence_length": 64,
    "vocab_size": 30000,
    "sequence_encoding": 'lstm',
    "final_dropout": 0.0,
    "lstm": {
        "layers": 1,
        "bidirectional": False,
        "dropout": 0,
        "hidden_dim": 512
    },
    "embeddings": {
        "mode": "bert_weights_extracted",
        "trainable": True,
        "dropout": 0.1
    }
})

model = SummaryModel(encoder_config=config_e, decoder_config=config_d)
model.to('cuda:0')

optimizer = torch.optim.AdamW(model.parameters(), lr=0.001)
trainer = SummaryTrainer(model, optimizer, criterions, epochs=12,  
                         #checkpoint_path='/content/drive/MyDrive/checkpoints/germasum/sent-cluster', 
                         tensorboard_path='/content/drive/MyDrive/experiments/metrics',
                         tensorboard_name='experiment-embeddings-1-v3',
                         tokenizer=tokenizer,
                         fp16=True)

trainer.train(train_set, test_set, eval=True, generation_size=16, from_epoch=0)

Some weights of the model checkpoint at bert-base-german-cased were not used when initializing BertModel: ['cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Using device: cuda
Running Epoch 1 ...


Epoch 1: 100%|██████████| 1884/1884 [02:58<00:00, 10.53steps/s, loss=5.87]



Epoch Loss: 5.867995778838224
Epoch Time Elapsed: 178.96014213562012


100%|██████████| 470/470 [00:29<00:00, 16.13steps/s, val_loss=4.97]


...


Validation Loss: 4.94587233106827
Reconstruction Rouge Scores: {'rouge-1': {'r': 0.1537860745950075, 'p': 0.1456340372186153, 'f': 0.14733939713009356}, 'rouge-2': {'r': 0.06394328152868525, 'p': 0.06121057118499574, 'f': 0.06207311137936844}, 'rouge-l': {'r': 0.1481883079723962, 'p': 0.14011463138865046, 'f': 0.14195178225327607}}
Summary Rouge Scores: {'rouge-1': {'r': 0.1375248963999521, 'p': 0.1322100848875049, 'f': 0.1317286192244759}, 'rouge-2': {'r': 0.06950510365836682, 'p': 0.06880166066547745, 'f': 0.06887182662034992}, 'rouge-l': {'r': 0.13244263545857118, 'p': 0.12693373491654925, 'f': 0.12668565511155136}}
Running Epoch 12 ...


Epoch 12: 100%|██████████| 1884/1884 [03:50<00:00,  8.17steps/s, loss=0.272]



Epoch Loss: 0.27222618150680106
Epoch Time Elapsed: 230.57194066047668


100%|██████████| 470/470 [00:54<00:00,  8.55steps/s, val_loss=4.98]



Validation Loss: 4.977923772727821
Reconstruction Rouge Scores: {'rouge-1': {'r': 0.15838603385167857, 'p': 0.15119815815185936, 'f': 0.15145820345119362}, 'rouge-2': {'r': 0.06402445035985409, 'p': 0.06144749566493264, 'f': 0.06201978704970494}, 'rouge-l': {'r': 0.1528698119842573, 'p': 0.14528884762515862, 'f': 0.1458184830685112}}
Summary Rouge Scores: {'rouge-1': {'r': 0.142919015532371, 'p': 0.1365242598792267, 'f': 0.13634943208719902}, 'rouge-2': {'r': 0.07163488380066169, 'p': 0.07116710382573273, 'f': 0.07111326514645337}, 'rouge-l': {'r': 0.13690089283854595, 'p': 0.13061998633132754, 'f': 0.1305729325660215}}
Total Time Elapsed: 3183.9393162727356


In [ ]:
# 2. bert full, trainable

config_e = ModelConfig({
    "embedding_dim": 768,
    "max_sequence_length": 64,
    "vocab_size": 30000,
    "sequence_encoding": 'lstm',
    "lstm": {
        "layers": 1,
        "bidirectional": False,
        "dropout": 0,
        "hidden_dim": 512
    },
    "embeddings": {
        "mode": "bert",
        "trainable": True,
        "dropout": 0.1
    }
})

config_d = ModelConfig({
    "embedding_dim": 768,
    "max_sequence_length": 64,
    "vocab_size": 30000,
    "sequence_encoding": 'lstm',
    "final_dropout": 0.0,
    "lstm": {
        "layers": 1,
        "bidirectional": False,
        "dropout": 0,
        "hidden_dim": 512
    },
    "embeddings": {
        "mode": "bert",
        "trainable": True,
        "dropout": 0.1
    }
})


model = SummaryModel(encoder_config=config_e, decoder_config=config_d)
model.to('cuda:0')

optimizer = torch.optim.AdamW(model.parameters(), lr=0.001)
trainer = SummaryTrainer(model, optimizer, criterions, epochs=4,  
                         #checkpoint_path='/content/drive/MyDrive/checkpoints/germasum/sent-cluster', 
                         tensorboard_path='/content/drive/MyDrive/experiments/metrics',
                         tensorboard_name='experiment-embeddings-2',
                         tokenizer=tokenizer,
                         fp16=True)

trainer.train(train_set, test_set, eval=True, generation_size=16, from_epoch=0)

Some weights of the model checkpoint at bert-base-german-cased were not used when initializing BertModel: ['cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Using device: cuda
Running Epoch 1 ...


Epoch 1: 100%|██████████| 1884/1884 [1:05:16<00:00,  2.08s/steps, loss=7.18]



Epoch Loss: 7.184495894392585
Epoch Time Elapsed: 3916.102871656418


100%|██████████| 470/470 [09:48<00:00,  1.25s/steps, val_loss=6.96]


...

Epoch 2: 100%|██████████| 1884/1884 [49:22<00:00,  1.57s/steps, loss=6.82]



Epoch Loss: 6.824593273823934
Epoch Time Elapsed: 2962.190970659256


100%|██████████| 470/470 [10:15<00:00,  1.31s/steps, val_loss=7.12]



Validation Loss: 7.124807717445049
Reconstruction Rouge Scores: {'rouge-1': {'r': 0.025446555214108, 'p': 0.10267857142857142, 'f': 0.039872976421143014}, 'rouge-2': {'r': 0.0, 'p': 0.0, 'f': 0.0}, 'rouge-l': {'r': 0.025446555214108, 'p': 0.10267857142857142, 'f': 0.039872976421143014}}
Summary Rouge Scores: {'rouge-1': {'r': 0.026366659963808887, 'p': 0.10213293650793652, 'f': 0.04057177420846034}, 'rouge-2': {'r': 0.0008427023707884036, 'p': 0.003125, 'f': 0.0013107334211018618}, 'rouge-l': {'r': 0.02624909869428683, 'p': 0.10138888888888889, 'f': 0.040369845878348645}}
Running Epoch 3 ...


Epoch 3:  98%|█████████▊| 1842/1884 [45:23<01:04,  1.54s/steps, loss=6.59]

In [ ]:
# 3. bert embedding layer, frozen
config_e = ModelConfig({
    "embedding_dim": 768,
    "max_sequence_length": 64,
    "vocab_size": 30000,
    "sequence_encoding": 'lstm',
    "lstm": {
        "layers": 1,
        "bidirectional": False,
        "dropout": 0,
        "hidden_dim": 512
    },
    "embeddings": {
        "mode": "bert_weights_extracted",
        "trainable": False,
        "dropout": 0.1
    }
})

config_d = ModelConfig({
    "embedding_dim": 768,
    "max_sequence_length": 64,
    "vocab_size": 30000,
    "sequence_encoding": 'lstm',
    "final_dropout": 0.0,
    "lstm": {
        "layers": 1,
        "bidirectional": False,
        "dropout": 0,
        "hidden_dim": 512
    },
    "embeddings": {
        "mode": "bert_weights_extracted",
        "trainable": False,
        "dropout": 0.1
    }
})


model = SummaryModel(encoder_config=config_e, decoder_config=config_d)
model.to('cuda:0')

optimizer = torch.optim.AdamW(model.parameters(), lr=0.001)
trainer = SummaryTrainer(model, optimizer, criterions, epochs=12,  
                         #checkpoint_path='/content/drive/MyDrive/checkpoints/germasum/sent-cluster', 
                         tensorboard_path='/content/drive/MyDrive/experiments/metrics',
                         tensorboard_name='experiment-embeddings-3',
                         tokenizer=tokenizer,
                         fp16=True)

trainer.train(train_set, test_set, eval=True, generation_size=16, from_epoch=0)

Some weights of the model checkpoint at bert-base-german-cased were not used when initializing BertModel: ['cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Using device: cuda
Running Epoch 1 ...


Epoch 1: 100%|██████████| 1884/1884 [02:03<00:00, 15.24steps/s, loss=6.27]



Epoch Loss: 6.266500405967236
Epoch Time Elapsed: 123.61331272125244


100%|██████████| 470/470 [00:21<00:00, 21.55steps/s, val_loss=5.45]


...


Validation Loss: 4.721883387242417
Reconstruction Rouge Scores: {'rouge-1': {'r': 0.14436646482721968, 'p': 0.13414036527689227, 'f': 0.13544819354701124}, 'rouge-2': {'r': 0.06040140415140415, 'p': 0.05848214285714286, 'f': 0.059179893652425056}, 'rouge-l': {'r': 0.1422420805907665, 'p': 0.13168884620772614, 'f': 0.13317221627103396}}
Summary Rouge Scores: {'rouge-1': {'r': 0.1101036803957551, 'p': 0.11196753564987894, 'f': 0.10859665850467876}, 'rouge-2': {'r': 0.045848785642471135, 'p': 0.04574064574074793, 'f': 0.04565930750085182}, 'rouge-l': {'r': 0.10387549372692369, 'p': 0.10511740636135068, 'f': 0.10226994086323762}}
Running Epoch 12 ...


Epoch 12: 100%|██████████| 1884/1884 [03:25<00:00,  9.15steps/s, loss=0.753]



Epoch Loss: 0.7531302158532543
Epoch Time Elapsed: 205.8667459487915


100%|██████████| 470/470 [00:49<00:00,  9.59steps/s, val_loss=4.79]



Validation Loss: 4.787601116956781
Reconstruction Rouge Scores: {'rouge-1': {'r': 0.14075115614638964, 'p': 0.13696967695727116, 'f': 0.13477313670224864}, 'rouge-2': {'r': 0.06040140415140415, 'p': 0.05818452380952381, 'f': 0.059026832428247945}, 'rouge-l': {'r': 0.1389654418606754, 'p': 0.1345887245763188, 'f': 0.13273232037571805}}
Summary Rouge Scores: {'rouge-1': {'r': 0.10758885693239795, 'p': 0.11194687513192479, 'f': 0.10697814412247197}, 'rouge-2': {'r': 0.04512620708662395, 'p': 0.04511705256699043, 'f': 0.04496870800606662}, 'rouge-l': {'r': 0.10267730425265846, 'p': 0.1060843328201452, 'f': 0.10178216541671645}}
Total Time Elapsed: 2820.54185962677


In [ ]:
# 4. bert full, frozen
config_e = ModelConfig({
    "embedding_dim": 768,
    "max_sequence_length": 64,
    "vocab_size": 30000,
    "sequence_encoding": 'lstm',
    "lstm": {
        "layers": 1,
        "bidirectional": False,
        "dropout": 0,
        "hidden_dim": 512
    },
    "embeddings": {
        "mode": "bert",
        "trainable": False,
        "dropout": 0.1
    }
})

config_d = ModelConfig({
    "embedding_dim": 768,
    "max_sequence_length": 64,
    "vocab_size": 30000,
    "sequence_encoding": 'lstm',
    "final_dropout": 0.0,
    "lstm": {
        "layers": 1,
        "bidirectional": False,
        "dropout": 0,
        "hidden_dim": 512
    },
    "embeddings": {
        "mode": "bert",
        "trainable": False,
        "dropout": 0.1
    }
})


model = SummaryModel(encoder_config=config_e, decoder_config=config_d)
model.to('cuda:0')

optimizer = torch.optim.AdamW(model.parameters(), lr=0.001)
trainer = SummaryTrainer(model, optimizer, criterions, epochs=12,  
                         #checkpoint_path='/content/drive/MyDrive/checkpoints/germasum/sent-cluster', 
                         tensorboard_path='/content/drive/MyDrive/experiments/metrics',
                         tensorboard_name='experiment-embeddings-4',
                         tokenizer=tokenizer,
                         fp16=True)

trainer.train(train_set, test_set, eval=True, generation_size=16, from_epoch=0)

Some weights of the model checkpoint at bert-base-german-cased were not used when initializing BertModel: ['cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Using device: cuda
Running Epoch 1 ...


Epoch 1: 100%|██████████| 1884/1884 [32:37<00:00,  1.04s/steps, loss=4.76]



Epoch Loss: 4.760971807135218
Epoch Time Elapsed: 1957.2149930000305


100%|██████████| 470/470 [10:17<00:00,  1.31s/steps, val_loss=3.56]


...


Epoch Loss: 0.9860197736052677
Epoch Time Elapsed: 3819.1227004528046


100%|██████████| 470/470 [29:02<00:00,  3.71s/steps, val_loss=2.63]



Validation Loss: 2.630052707971716
Reconstruction Rouge Scores: {'rouge-1': {'r': 0.023942413220835568, 'p': 0.1351190476190476, 'f': 0.0399487356365121}, 'rouge-2': {'r': 0.0, 'p': 0.0, 'f': 0.0}, 'rouge-l': {'r': 0.023942413220835568, 'p': 0.1351190476190476, 'f': 0.0399487356365121}}
Summary Rouge Scores: {'rouge-1': {'r': 0.02091285206254333, 'p': 0.09972222222222232, 'f': 0.033026929352357536}, 'rouge-2': {'r': 0.00025886422945246477, 'p': 0.0007341269841269842, 'f': 0.00037489665136885273}, 'rouge-l': {'r': 0.020852727002418275, 'p': 0.09942460317460328, 'f': 0.03292702925245743}}
Running Epoch 5 ...


Epoch 5:   8%|▊         | 154/1884 [05:04<57:06,  1.98s/steps, loss=0.903]  


KeyboardInterrupt: ignored

## Experiment 2: LSTM dimension
1. hidden 256
2. hidden 512
3. hidden 1024

In [ ]:
# 1. 256
config_e = ModelConfig({
    "embedding_dim": 768,
    "max_sequence_length": 64,
    "vocab_size": 30000,
    "sequence_encoding": 'lstm',
    "lstm": {
        "layers": 1,
        "bidirectional": False,
        "dropout": 0,
        "hidden_dim": 256
    },
    "embeddings": {
        "mode": "bert_weights_extracted",
        "trainable": True,
        "dropout": 0.1
    }
})

config_d = ModelConfig({
    "embedding_dim": 768,
    "max_sequence_length": 64,
    "vocab_size": 30000,
    "sequence_encoding": 'lstm',
    "final_dropout": 0.0,
    "lstm": {
        "layers": 1,
        "bidirectional": False,
        "dropout": 0,
        "hidden_dim": 256
    },
    "embeddings": {
        "mode": "bert_weights_extracted",
        "trainable": True,
        "dropout": 0.1
    }
})


model = SummaryModel(encoder_config=config_e, decoder_config=config_d)
model.to('cuda:0')

optimizer = torch.optim.AdamW(model.parameters(), lr=0.001)
trainer = SummaryTrainer(model, optimizer, criterions, epochs=12,  
                         #checkpoint_path='/content/drive/MyDrive/checkpoints/germasum/sent-cluster', 
                         tensorboard_path='/content/drive/MyDrive/experiments/metrics',
                         tensorboard_name='experiment-lstm-dim-1-v2',
                         tokenizer=tokenizer,
                         fp16=True)

trainer.train(train_set, test_set, eval=True, generation_size=16, from_epoch=0)

Some weights of the model checkpoint at bert-base-german-cased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Using device: cuda
Running Epoch 1 ...


Epoch 1: 100%|██████████| 1884/1884 [01:26<00:00, 21.83steps/s, loss=6.43]



Epoch Loss: 6.43330778553291
Epoch Time Elapsed: 86.31449127197266


100%|██████████| 470/470 [00:16<00:00, 27.98steps/s, val_loss=5.5]


...


Validation Loss: 4.333678342511903
Reconstruction Rouge Scores: {'rouge-1': {'r': 0.24379191554329893, 'p': 0.2406899488808486, 'f': 0.24067337057074387}, 'rouge-2': {'r': 0.1709090909090909, 'p': 0.17277777777777778, 'f': 0.17173616616125795}, 'rouge-l': {'r': 0.23713191563786293, 'p': 0.2346483651452993, 'f': 0.2343657011188401}}
Summary Rouge Scores: {'rouge-1': {'r': 0.12149472816120498, 'p': 0.11282608879307324, 'f': 0.11373940290386607}, 'rouge-2': {'r': 0.05467203037218271, 'p': 0.05293193108444271, 'f': 0.0531202420625885}, 'rouge-l': {'r': 0.11509696334767953, 'p': 0.10700585642175253, 'f': 0.10781033275165892}}
Running Epoch 12 ...


Epoch 12: 100%|██████████| 1884/1884 [02:55<00:00, 10.77steps/s, loss=1.55]



Epoch Loss: 1.554893852951798
Epoch Time Elapsed: 175.01007771492004


100%|██████████| 470/470 [00:42<00:00, 10.97steps/s, val_loss=4.35]



Validation Loss: 4.34786576523443
Reconstruction Rouge Scores: {'rouge-1': {'r': 0.2518963810494361, 'p': 0.2490192640961587, 'f': 0.2487584709621166}, 'rouge-2': {'r': 0.1786810411810412, 'p': 0.1788888888888889, 'f': 0.17853510788043594}, 'rouge-l': {'r': 0.249847309431399, 'p': 0.24659252416941874, 'f': 0.24655225784259077}}
Summary Rouge Scores: {'rouge-1': {'r': 0.11595765622237161, 'p': 0.11205165791457834, 'f': 0.11099677421740205}, 'rouge-2': {'r': 0.05557637701401348, 'p': 0.056114031821667375, 'f': 0.05533385013876686}, 'rouge-l': {'r': 0.1109999558700134, 'p': 0.10716780105119836, 'f': 0.10622360719945172}}
Total Time Elapsed: 2387.9292693138123


In [ ]:
# 2. 512
config_e = ModelConfig({
    "embedding_dim": 768,
    "max_sequence_length": 64,
    "vocab_size": 30000,
    "sequence_encoding": 'lstm',
    "lstm": {
        "layers": 1,
        "bidirectional": False,
        "dropout": 0,
        "hidden_dim": 512
    },
    "embeddings": {
        "mode": "bert_weights_extracted",
        "trainable": True,
        "dropout": 0.1
    }
})

config_d = ModelConfig({
    "embedding_dim": 768,
    "max_sequence_length": 64,
    "vocab_size": 30000,
    "sequence_encoding": 'lstm',
    "final_dropout": 0.0,
    "lstm": {
        "layers": 1,
        "bidirectional": False,
        "dropout": 0,
        "hidden_dim": 512
    },
    "embeddings": {
        "mode": "bert_weights_extracted",
        "trainable": True,
        "dropout": 0.1
    }
})

model = SummaryModel(encoder_config=config_e, decoder_config=config_d)
model.to('cuda:0')

optimizer = torch.optim.AdamW(model.parameters(), lr=0.001)
trainer = SummaryTrainer(model, optimizer, criterions, epochs=12,  
                         #checkpoint_path='/content/drive/MyDrive/checkpoints/germasum/sent-cluster', 
                         tensorboard_path='/content/drive/MyDrive/experiments/metrics',
                         tensorboard_name='experiment-lstm-dim-2-v2',
                         tokenizer=tokenizer,
                         fp16=True)

trainer.train(train_set, test_set, eval=True, generation_size=16, from_epoch=0)

Some weights of the model checkpoint at bert-base-german-cased were not used when initializing BertModel: ['cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Using device: cuda
Running Epoch 1 ...


Epoch 1: 100%|██████████| 1884/1884 [02:18<00:00, 13.61steps/s, loss=5.9]



Epoch Loss: 5.8951622841685944
Epoch Time Elapsed: 138.39443588256836


100%|██████████| 470/470 [00:35<00:00, 13.39steps/s, val_loss=5]


...


Validation Loss: 4.901987302618188
Reconstruction Rouge Scores: {'rouge-1': {'r': 0.29690380921816195, 'p': 0.2930291346037241, 'f': 0.2917249809481885}, 'rouge-2': {'r': 0.22788410419989366, 'p': 0.22856125356125354, 'f': 0.22785278501024722}, 'rouge-l': {'r': 0.2950470452924324, 'p': 0.29151398308857257, 'f': 0.2901190052245657}}
Summary Rouge Scores: {'rouge-1': {'r': 0.11297239988222801, 'p': 0.11194026273991126, 'f': 0.10895001625492831}, 'rouge-2': {'r': 0.04779182940346097, 'p': 0.048020980878251136, 'f': 0.04770074929673359}, 'rouge-l': {'r': 0.10746673549125217, 'p': 0.10587825433128481, 'f': 0.10339499932800021}}
Running Epoch 12 ...


Epoch 12: 100%|██████████| 1884/1884 [03:56<00:00,  7.98steps/s, loss=0.289]



Epoch Loss: 0.28894069788685167
Epoch Time Elapsed: 236.02952814102173


100%|██████████| 470/470 [00:58<00:00,  7.98steps/s, val_loss=4.94]



Validation Loss: 4.943694781330547
Reconstruction Rouge Scores: {'rouge-1': {'r': 0.3157994968638495, 'p': 0.31980683272982036, 'f': 0.3135223646147268}, 'rouge-2': {'r': 0.24407777164356112, 'p': 0.2507407407407407, 'f': 0.24545555025095026}, 'rouge-l': {'r': 0.31394273293812003, 'p': 0.31776720569019334, 'f': 0.3115867185614337}}
Summary Rouge Scores: {'rouge-1': {'r': 0.11834160909946734, 'p': 0.11807418379816484, 'f': 0.1144212899557263}, 'rouge-2': {'r': 0.050220459103216285, 'p': 0.05091741340681658, 'f': 0.05030590733976116}, 'rouge-l': {'r': 0.11338087116334408, 'p': 0.11275040335995078, 'f': 0.10946182954472727}}
Total Time Elapsed: 3185.4256105422974


In [ ]:
# 3. 1024
config_e = ModelConfig({
    "embedding_dim": 768,
    "max_sequence_length": 64,
    "vocab_size": 30000,
    "sequence_encoding": 'lstm',
    "lstm": {
        "layers": 1,
        "bidirectional": False,
        "dropout": 0,
        "hidden_dim": 1024
    },
    "embeddings": {
        "mode": "bert_weights_extracted",
        "trainable": True,
        "dropout": 0.1
    }
})

config_d = ModelConfig({
    "embedding_dim": 768,
    "max_sequence_length": 64,
    "vocab_size": 30000,
    "sequence_encoding": 'lstm',
    "final_dropout": 0.0,
    "lstm": {
        "layers": 1,
        "bidirectional": False,
        "dropout": 0,
        "hidden_dim": 1024
    },
    "embeddings": {
        "mode": "bert_weights_extracted",
        "trainable": True,
        "dropout": 0.1
    }
})


model = SummaryModel(encoder_config=config_e, decoder_config=config_d)
model.to('cuda:0')

optimizer = torch.optim.AdamW(model.parameters(), lr=0.001)
trainer = SummaryTrainer(model, optimizer, criterions, epochs=12,  
                         #checkpoint_path='/content/drive/MyDrive/checkpoints/germasum/sent-cluster', 
                         tensorboard_path='/content/drive/MyDrive/experiments/metrics',
                         tensorboard_name='experiment-lstm-dim-3-v2',
                         tokenizer=tokenizer,
                         fp16=True)

trainer.train(train_set, test_set, eval=True, generation_size=16, from_epoch=0)

Some weights of the model checkpoint at bert-base-german-cased were not used when initializing BertModel: ['cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Using device: cuda
Running Epoch 1 ...


Epoch 1: 100%|██████████| 1884/1884 [02:56<00:00, 10.68steps/s, loss=5.73]



Epoch Loss: 5.734188528703783
Epoch Time Elapsed: 176.36219453811646


100%|██████████| 470/470 [01:00<00:00,  7.78steps/s, val_loss=4.88]


...


Validation Loss: 5.006263398295681
Reconstruction Rouge Scores: {'rouge-1': {'r': 0.1626044519641279, 'p': 0.15976279751128672, 'f': 0.15742341196774703}, 'rouge-2': {'r': 0.06538160769527229, 'p': 0.06578153988868275, 'f': 0.06488928021514366}, 'rouge-l': {'r': 0.15668854084233758, 'p': 0.15349652837737057, 'f': 0.15148096278399664}}
Summary Rouge Scores: {'rouge-1': {'r': 0.13180640165947594, 'p': 0.12621416831627102, 'f': 0.1260122953388423}, 'rouge-2': {'r': 0.06273699612079957, 'p': 0.062442582782181316, 'f': 0.062351961484958446}, 'rouge-l': {'r': 0.12470555178440926, 'p': 0.11944727241702346, 'f': 0.11933301233789982}}
Running Epoch 12 ...


Epoch 12: 100%|██████████| 1884/1884 [04:13<00:00,  7.43steps/s, loss=0.278]



Epoch Loss: 0.27750714445406754
Epoch Time Elapsed: 253.43056440353394


100%|██████████| 470/470 [00:57<00:00,  8.12steps/s, val_loss=5.02]



Validation Loss: 5.018720442507826
Reconstruction Rouge Scores: {'rouge-1': {'r': 0.17728224618276484, 'p': 0.17131851461155775, 'f': 0.17152344592808744}, 'rouge-2': {'r': 0.06811286324096884, 'p': 0.06740126606198035, 'f': 0.06736439249982158}, 'rouge-l': {'r': 0.17031874239044334, 'p': 0.16401779650920928, 'f': 0.16454612667398244}}
Summary Rouge Scores: {'rouge-1': {'r': 0.13439002038150558, 'p': 0.12745554505855877, 'f': 0.12785291178353617}, 'rouge-2': {'r': 0.06441976267723717, 'p': 0.0640232587017091, 'f': 0.06402032094003918}, 'rouge-l': {'r': 0.12845305025964876, 'p': 0.12189680212076041, 'f': 0.12229799205408483}}
Total Time Elapsed: 3626.977273464203


## Experiment 3: LSTM bidirectional
1. unidirectional
2. bidirectional

In [ ]:
# 1. unidirectional
config_e = ModelConfig({
    "embedding_dim": 768,
    "max_sequence_length": 64,
    "vocab_size": 30000,
    "sequence_encoding": 'lstm',
    "lstm": {
        "layers": 1,
        "bidirectional": False,
        "dropout": 0,
        "hidden_dim": 512
    },
    "embeddings": {
        "mode": "bert_weights_extracted",
        "trainable": True,
        "dropout": 0.1
    }
})

config_d = ModelConfig({
    "embedding_dim": 768,
    "max_sequence_length": 64,
    "vocab_size": 30000,
    "sequence_encoding": 'lstm',
    "final_dropout": 0.0,
    "lstm": {
        "layers": 1,
        "bidirectional": False,
        "dropout": 0,
        "hidden_dim": 512
    },
    "embeddings": {
        "mode": "bert_weights_extracted",
        "trainable": True,
        "dropout": 0.1
    }
})


model = SummaryModel(encoder_config=config_e, decoder_config=config_d)
model.to('cuda:0')

optimizer = torch.optim.AdamW(model.parameters(), lr=0.001)
trainer = SummaryTrainer(model, optimizer, criterions, epochs=12,  
                         #checkpoint_path='/content/drive/MyDrive/checkpoints/germasum/sent-cluster', 
                         tensorboard_path='/content/drive/MyDrive/experiments/metrics',
                         tensorboard_name='experiment-lstm-bidir-1-v3',
                         tokenizer=tokenizer,
                         fp16=True)

trainer.train(train_set, test_set, eval=True, generation_size=16, from_epoch=0)

Some weights of the model checkpoint at bert-base-german-cased were not used when initializing BertModel: ['cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Using device: cuda
Running Epoch 1 ...


Epoch 1: 100%|██████████| 1884/1884 [02:16<00:00, 13.84steps/s, loss=5.88]



Epoch Loss: 5.876870083913302
Epoch Time Elapsed: 136.150249004364


100%|██████████| 470/470 [00:34<00:00, 13.47steps/s, val_loss=4.97]


...


Validation Loss: 4.856419398147233
Reconstruction Rouge Scores: {'rouge-1': {'r': 0.3096584901341404, 'p': 0.3140587689342485, 'f': 0.3091694660744493}, 'rouge-2': {'r': 0.24237322618901566, 'p': 0.2436769943019943, 'f': 0.24272042678299022}, 'rouge-l': {'r': 0.3083764388520891, 'p': 0.31307837677738576, 'f': 0.30805835496333817}}
Summary Rouge Scores: {'rouge-1': {'r': 0.12653167792723835, 'p': 0.11541693851009932, 'f': 0.11736525579596858}, 'rouge-2': {'r': 0.05748454855420834, 'p': 0.05458878212862995, 'f': 0.05542946747351484}, 'rouge-l': {'r': 0.1194793536129895, 'p': 0.10928212270964233, 'f': 0.11106186832604172}}
Running Epoch 12 ...


Epoch 12: 100%|██████████| 1884/1884 [03:54<00:00,  8.03steps/s, loss=0.287]



Epoch Loss: 0.2873283103474521
Epoch Time Elapsed: 234.60566926002502


100%|██████████| 470/470 [01:00<00:00,  7.72steps/s, val_loss=4.92]



Validation Loss: 4.916514934591261
Reconstruction Rouge Scores: {'rouge-1': {'r': 0.30484798347056363, 'p': 0.2988109096858342, 'f': 0.29935023684648754}, 'rouge-2': {'r': 0.22930504437083385, 'p': 0.2308992165242165, 'f': 0.22980162699488973}, 'rouge-l': {'r': 0.30484798347056363, 'p': 0.2988109096858342, 'f': 0.29935023684648754}}
Summary Rouge Scores: {'rouge-1': {'r': 0.12766898776893618, 'p': 0.11667654274679866, 'f': 0.11847834313364983}, 'rouge-2': {'r': 0.05516687313809391, 'p': 0.05415847724768739, 'f': 0.05432664424204815}, 'rouge-l': {'r': 0.12113315248181662, 'p': 0.11112367239522514, 'f': 0.11272949213415193}}
Total Time Elapsed: 3148.843024969101


In [ ]:
# 2. bidirectional
config_e = ModelConfig({
    "embedding_dim": 768,
    "max_sequence_length": 64,
    "vocab_size": 30000,
    "sequence_encoding": 'lstm',
    "lstm": {
        "layers": 1,
        "bidirectional": True,
        "dropout": 0,
        "hidden_dim": 512
    },
    "embeddings": {
        "mode": "bert_weights_extracted",
        "trainable": True,
        "dropout": 0.1
    }
})

config_d = ModelConfig({
    "embedding_dim": 768,
    "max_sequence_length": 64,
    "vocab_size": 30000,
    "sequence_encoding": 'lstm',
    "final_dropout": 0.0,
    "lstm": {
        "layers": 1,
        "bidirectional": False,
        "dropout": 0,
        "hidden_dim": 1024
    },
    "embeddings": {
        "mode": "bert_weights_extracted",
        "trainable": True,
        "dropout": 0.1
    }
})

model = SummaryModel(encoder_config=config_e, decoder_config=config_d)
model.to('cuda:0')

optimizer = torch.optim.AdamW(model.parameters(), lr=0.001)
trainer = SummaryTrainer(model, optimizer, criterions, epochs=12,  
                         #checkpoint_path='/content/drive/MyDrive/checkpoints/germasum/sent-cluster', 
                         tensorboard_path='/content/drive/MyDrive/experiments/metrics',
                         tensorboard_name='experiment-lstm-bidir-2-v2',
                         tokenizer=tokenizer,
                         fp16=True)

trainer.train(train_set, test_set, eval=True, generation_size=16, from_epoch=0)

Some weights of the model checkpoint at bert-base-german-cased were not used when initializing BertModel: ['cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Using device: cuda
Running Epoch 1 ...


Epoch 1: 100%|██████████| 1884/1884 [03:54<00:00,  8.05steps/s, loss=5.71]



Epoch Loss: 5.706625656121966
Epoch Time Elapsed: 234.04500365257263


100%|██████████| 470/470 [00:58<00:00,  8.05steps/s, val_loss=4.73]


...


Validation Loss: 4.690639368810114
Reconstruction Rouge Scores: {'rouge-1': {'r': 0.370728922012217, 'p': 0.3713074517163861, 'f': 0.37029165492987703}, 'rouge-2': {'r': 0.2841546538257064, 'p': 0.28580537746216106, 'f': 0.2847777612898213}, 'rouge-l': {'r': 0.3665812246190835, 'p': 0.36707247730718306, 'f': 0.36613141597040944}}
Summary Rouge Scores: {'rouge-1': {'r': 0.11943525326503363, 'p': 0.11894693210734258, 'f': 0.11648803942053358}, 'rouge-2': {'r': 0.051713967908972126, 'p': 0.0526006477967502, 'f': 0.05148757155847852}, 'rouge-l': {'r': 0.11331597162640077, 'p': 0.1123834218412851, 'f': 0.11027526202119056}}
Running Epoch 12 ...


Epoch 12: 100%|██████████| 1884/1884 [03:50<00:00,  8.16steps/s, loss=0.208]



Epoch Loss: 0.20846895972019194
Epoch Time Elapsed: 230.97923350334167


100%|██████████| 470/470 [00:53<00:00,  8.74steps/s, val_loss=4.75]



Validation Loss: 4.7529485297679885
Reconstruction Rouge Scores: {'rouge-1': {'r': 0.3669814054805469, 'p': 0.36602599088201676, 'f': 0.3657593800057022}, 'rouge-2': {'r': 0.27945622725227987, 'p': 0.27925815805272514, 'f': 0.2792601286856817}, 'rouge-l': {'r': 0.36426837776751925, 'p': 0.3634027661535289, 'f': 0.36309481067446625}}
Summary Rouge Scores: {'rouge-1': {'r': 0.12068015568589979, 'p': 0.12388858229983793, 'f': 0.11967021678301873}, 'rouge-2': {'r': 0.05120356891338548, 'p': 0.051734078755088224, 'f': 0.050845707583079325}, 'rouge-l': {'r': 0.11414916820473757, 'p': 0.11689297593521457, 'f': 0.11308559107505596}}
Total Time Elapsed: 3523.9703443050385


## Experiment 4: Dropout layer
1. 0.00
2. 0.25
3. 0.50

In [ ]:
# 1. 0.00
config_e = ModelConfig({
    "embedding_dim": 768,
    "max_sequence_length": 64,
    "vocab_size": 30000,
    "sequence_encoding": 'lstm',
    "lstm": {
        "layers": 1,
        "bidirectional": True,
        "dropout": 0,
        "hidden_dim": 512
    },
    "embeddings": {
        "mode": "bert_weights_extracted",
        "trainable": True,
        "dropout": 0.1
    }
})

config_d = ModelConfig({
    "embedding_dim": 768,
    "max_sequence_length": 64,
    "vocab_size": 30000,
    "sequence_encoding": 'lstm',
    "final_dropout": 0.0,
    "lstm": {
        "layers": 1,
        "bidirectional": False,
        "dropout": 0,
        "hidden_dim": 1024
    },
    "embeddings": {
        "mode": "bert_weights_extracted",
        "trainable": True,
        "dropout": 0.1
    }
})

model = SummaryModel(encoder_config=config_e, decoder_config=config_d)
model.to('cuda:0')

optimizer = torch.optim.AdamW(model.parameters(), lr=0.001)
trainer = SummaryTrainer(model, optimizer, criterions, epochs=12,  
                         #checkpoint_path='/content/drive/MyDrive/checkpoints/germasum/sent-cluster', 
                         tensorboard_path='/content/drive/MyDrive/experiments/metrics',
                         tensorboard_name='experiment-dropout-1',
                         tokenizer=tokenizer,
                         fp16=True)

trainer.train(train_set, test_set, eval=True, generation_size=16, from_epoch=0)

Some weights of the model checkpoint at bert-base-german-cased were not used when initializing BertModel: ['cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Using device: cuda
Running Epoch 1 ...


Epoch 1: 100%|██████████| 1884/1884 [03:07<00:00, 10.03steps/s, loss=5.8]



Epoch Loss: 5.798809995360656
Epoch Time Elapsed: 187.86423206329346


100%|██████████| 470/470 [00:48<00:00,  9.65steps/s, val_loss=4.87]


...


Validation Loss: 4.785969335944983
Reconstruction Rouge Scores: {'rouge-1': {'r': 0.22122358291559316, 'p': 0.22489837026928036, 'f': 0.22217076947136452}, 'rouge-2': {'r': 0.12113781882636294, 'p': 0.12064980158730158, 'f': 0.12080730979726215}, 'rouge-l': {'r': 0.21419478632131142, 'p': 0.21773212514804344, 'f': 0.21510908532102804}}
Summary Rouge Scores: {'rouge-1': {'r': 0.11215855768565892, 'p': 0.11038550810970565, 'f': 0.10883661279955198}, 'rouge-2': {'r': 0.04972372513431715, 'p': 0.048835987376687456, 'f': 0.0489536842572315}, 'rouge-l': {'r': 0.10625809357132646, 'p': 0.1043153751919626, 'f': 0.10301887740069415}}
Running Epoch 12 ...


Epoch 12: 100%|██████████| 1884/1884 [03:55<00:00,  7.98steps/s, loss=0.219]



Epoch Loss: 0.21903314296663293
Epoch Time Elapsed: 235.96450328826904


100%|██████████| 470/470 [00:52<00:00,  9.00steps/s, val_loss=4.83]



Validation Loss: 4.827534064077009
Reconstruction Rouge Scores: {'rouge-1': {'r': 0.20896623654404076, 'p': 0.21675345313017053, 'f': 0.21184400308697646}, 'rouge-2': {'r': 0.11949996392763848, 'p': 0.12260521655620586, 'f': 0.12078978754474734}, 'rouge-l': {'r': 0.20321534923259438, 'p': 0.2108804446018164, 'f': 0.20605768493652787}}
Summary Rouge Scores: {'rouge-1': {'r': 0.10926964146383826, 'p': 0.10757333798858097, 'f': 0.10608304743448331}, 'rouge-2': {'r': 0.04697482269227714, 'p': 0.04699968783535048, 'f': 0.04669646537597305}, 'rouge-l': {'r': 0.10312025845160257, 'p': 0.10114896931816321, 'f': 0.09991643000941862}}
Total Time Elapsed: 3489.3736774921417


In [ ]:
# 2. 0.25
config_e = ModelConfig({
    "embedding_dim": 768,
    "max_sequence_length": 64,
    "vocab_size": 30000,
    "sequence_encoding": 'lstm',
    "lstm": {
        "layers": 1,
        "bidirectional": True,
        "dropout": 0,
        "hidden_dim": 512
    },
    "embeddings": {
        "mode": "bert_weights_extracted",
        "trainable": True,
        "dropout": 0.1
    }
})

config_d = ModelConfig({
    "embedding_dim": 768,
    "max_sequence_length": 64,
    "vocab_size": 30000,
    "sequence_encoding": 'lstm',
    "final_dropout": 0.25,
    "lstm": {
        "layers": 1,
        "bidirectional": False,
        "dropout": 0,
        "hidden_dim": 1024
    },
    "embeddings": {
        "mode": "bert_weights_extracted",
        "trainable": True,
        "dropout": 0.1
    }
})

model = SummaryModel(encoder_config=config_e, decoder_config=config_d)
model.to('cuda:0')

optimizer = torch.optim.AdamW(model.parameters(), lr=0.001)
trainer = SummaryTrainer(model, optimizer, criterions, epochs=12,  
                         #checkpoint_path='/content/drive/MyDrive/checkpoints/germasum/sent-cluster', 
                         tensorboard_path='/content/drive/MyDrive/experiments/metrics',
                         tensorboard_name='experiment-dropout-2',
                         tokenizer=tokenizer,
                         fp16=True)

trainer.train(train_set, test_set, eval=True, generation_size=16, from_epoch=0)

Some weights of the model checkpoint at bert-base-german-cased were not used when initializing BertModel: ['cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Using device: cuda
Running Epoch 1 ...


Epoch 1: 100%|██████████| 1884/1884 [02:52<00:00, 10.93steps/s, loss=5.86]



Epoch Loss: 5.856285767285687
Epoch Time Elapsed: 172.33866047859192


100%|██████████| 470/470 [00:54<00:00,  8.57steps/s, val_loss=4.92]


...


Validation Loss: 4.421560757361226
Reconstruction Rouge Scores: {'rouge-1': {'r': 0.2728835621989384, 'p': 0.28037800704231797, 'f': 0.27398421547763385}, 'rouge-2': {'r': 0.16407295404662908, 'p': 0.16890426047237478, 'f': 0.165465016960219}, 'rouge-l': {'r': 0.2694080678032791, 'p': 0.27665854198356715, 'f': 0.2703994307501146}}
Summary Rouge Scores: {'rouge-1': {'r': 0.15164098937763354, 'p': 0.144653921889129, 'f': 0.1446595031553387}, 'rouge-2': {'r': 0.07807179881850777, 'p': 0.07669300147085657, 'f': 0.0767752104464318}, 'rouge-l': {'r': 0.1460307519002251, 'p': 0.13908525047773224, 'f': 0.13925239839395534}}
Running Epoch 12 ...


Epoch 12: 100%|██████████| 1884/1884 [04:08<00:00,  7.59steps/s, loss=0.329]



Epoch Loss: 0.3286151202433591
Epoch Time Elapsed: 248.2452688217163


100%|██████████| 470/470 [00:59<00:00,  7.96steps/s, val_loss=4.4]



Validation Loss: 4.401616746606077
Reconstruction Rouge Scores: {'rouge-1': {'r': 0.30782781340351745, 'p': 0.29470514841196654, 'f': 0.2987950938963505}, 'rouge-2': {'r': 0.20610802186563607, 'p': 0.19319473993429098, 'f': 0.19811220947309457}, 'rouge-l': {'r': 0.3041366530313226, 'p': 0.29098918645247007, 'f': 0.29509534350071}}
Summary Rouge Scores: {'rouge-1': {'r': 0.16179536471743028, 'p': 0.1518028460981215, 'f': 0.15304088308468583}, 'rouge-2': {'r': 0.08329328481839403, 'p': 0.08078679524973857, 'f': 0.08131192064848122}, 'rouge-l': {'r': 0.15556428054240767, 'p': 0.1455159885938466, 'f': 0.14695211711758593}}
Total Time Elapsed: 3593.28471660614


In [ ]:
# 3. 0.50
config_e = ModelConfig({
    "embedding_dim": 768,
    "max_sequence_length": 64,
    "vocab_size": 30000,
    "sequence_encoding": 'lstm',
    "lstm": {
        "layers": 1,
        "bidirectional": True,
        "dropout": 0,
        "hidden_dim": 512
    },
    "embeddings": {
        "mode": "bert_weights_extracted",
        "trainable": True,
        "dropout": 0.1
    }
})

config_d = ModelConfig({
    "embedding_dim": 768,
    "max_sequence_length": 64,
    "vocab_size": 30000,
    "sequence_encoding": 'lstm',
    "final_dropout": 0.5,
    "lstm": {
        "layers": 1,
        "bidirectional": False,
        "dropout": 0,
        "hidden_dim": 1024
    },
    "embeddings": {
        "mode": "bert_weights_extracted",
        "trainable": True,
        "dropout": 0.1
    }
})

model = SummaryModel(encoder_config=config_e, decoder_config=config_d)
model.to('cuda:0')

optimizer = torch.optim.AdamW(model.parameters(), lr=0.001)
trainer = SummaryTrainer(model, optimizer, criterions, epochs=12,  
                         #checkpoint_path='/content/drive/MyDrive/checkpoints/germasum/sent-cluster', 
                         tensorboard_path='/content/drive/MyDrive/experiments/metrics',
                         tensorboard_name='experiment-dropout-3',
                         tokenizer=tokenizer,
                         fp16=True)

trainer.train(train_set, test_set, eval=True, generation_size=16, from_epoch=0)

Some weights of the model checkpoint at bert-base-german-cased were not used when initializing BertModel: ['cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Using device: cuda
Running Epoch 1 ...


Epoch 1: 100%|██████████| 1884/1884 [02:54<00:00, 10.77steps/s, loss=6.09]



Epoch Loss: 6.08863532407933
Epoch Time Elapsed: 174.8703591823578


100%|██████████| 470/470 [00:49<00:00,  9.55steps/s, val_loss=5.17]


...


Validation Loss: 4.375317615215477
Reconstruction Rouge Scores: {'rouge-1': {'r': 0.17983408596616132, 'p': 0.18039097748022895, 'f': 0.17925389377728762}, 'rouge-2': {'r': 0.09533822587005816, 'p': 0.09379524537087562, 'f': 0.09452504738836601}, 'rouge-l': {'r': 0.1749980473528149, 'p': 0.1755690487087203, 'f': 0.17444791078562846}}
Summary Rouge Scores: {'rouge-1': {'r': 0.13697286477053966, 'p': 0.13339091148106716, 'f': 0.13243839487907605}, 'rouge-2': {'r': 0.06585141993663733, 'p': 0.06558900046180177, 'f': 0.0652898030676723}, 'rouge-l': {'r': 0.13141945966526114, 'p': 0.12774581168862534, 'f': 0.12700170086017065}}
Running Epoch 12 ...


Epoch 12: 100%|██████████| 1884/1884 [03:57<00:00,  7.92steps/s, loss=0.83]



Epoch Loss: 0.8301080515609782
Epoch Time Elapsed: 237.81420612335205


100%|██████████| 470/470 [00:51<00:00,  9.18steps/s, val_loss=4.41]



Validation Loss: 4.407155112654313
Reconstruction Rouge Scores: {'rouge-1': {'r': 0.1698733301020441, 'p': 0.17307088625036357, 'f': 0.17058313271468745}, 'rouge-2': {'r': 0.07823429377998077, 'p': 0.07829861111111111, 'f': 0.07823316918058984}, 'rouge-l': {'r': 0.16573808904910117, 'p': 0.16877854431358888, 'f': 0.16639578008462072}}
Summary Rouge Scores: {'rouge-1': {'r': 0.14187468418081128, 'p': 0.13853558151455525, 'f': 0.1371709417153682}, 'rouge-2': {'r': 0.07091692880036392, 'p': 0.07167550751828547, 'f': 0.07073299360019882}, 'rouge-l': {'r': 0.13653834706552848, 'p': 0.1328389469337278, 'f': 0.13178652186281073}}
Total Time Elapsed: 3422.5359632968903
